# Generic Backtracking Algorithm

All the code for this assessment is the module `lib_assessment_1_STOR_i.py` in order to improve the readability of this notebook. The module should be in the same directory as this notebook to run properly.

In the following code cell we import the corresponding module.

In [1]:
import lib_assessment_1_STOR_i_609
from lib_assessment_1_STOR_i_609 import * 

For *Typing Hint* the only dependency of this module is `typing`. Besides that we only use base `Python 3.11.4`. 

In the following cell, we show the `backtrack` generic function that we will use to solve both assessment problems.

In [2]:
#################################
# Generic Backtracking function #
#################################
def backtrack(P: Type, c) -> None:
    """Generic backtrack function

    Args:
        P (Class): A Class that implements the problem to solve
        c (_type_): Format of the candidates corresponding to the class P
    """
    # Stopping conditions
    if P.reject(c):
        return
    if P.accept(c):
        P.output(c)
        return
    s = P.first(c)  # descendent
    backtrack(P, s)  # Recur on descendent
    s = P.next(c)  # brother node
    backtrack(P, s)  # Recur on brother node

In the following cell we show that this function definitely works. We will explain the `Partition` class later in the report. 

In [3]:
P_example = Partition(5)
backtrack(P_example,[[],5,5])
P_example.partitions

[[5], [4, 1], [3, 2], [3, 1, 1], [2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1]]

This implementation is a little bit different from the one suggested in https://en.wikipedia.org/wiki/Backtracking. The main difference is that we avoid to do a loop in order to be more efficient and reduce the function calls (Python and R are languages that particularly struggle with recursive function calls). 

Here `P` is a python class that abstracts the problem to solve through the *backtracking algorithm* and `c` is the current to state explore. Now we describe briefly each of the methods that the class or problem `P` should have. 
1. `reject`: Returns `True` if that candidate `c` leads to an infeasible solution, otherwise returns `False`
2. `accept`: Return `True` if candidate `c` is a solution, otherwise returns `False`.
3. The `output` method saves the accepted candidate in a class attribute of `P`.

We could think backtraking as a way to come back in the tree of possible candidates that conform the problem. 

Given a candidate `c`, the methods `first` and `next` moves to an adjacent candidate in the following way:
1. `first` method generated the next candidate or child node corresponding to the input. This corresponds to a *depth search*. 
2. After a depth serch, the recursion come back to the current candidate and the `next` method obtains the brother node corresponding to that extension. This could be interpreted as a *breadth search*.

In a nutshell:
- The implementation of backtracking presented use a depth and breadth search approach to reduce function calls. 
- The abstract tree of candidate and all the methods presented here should be coded in a class `P` that defines the problem. Given that class with all of this methods and the heuristic search, the code is *reusable*.

Finally, in order to solve both problems we incorporate in the class `P` the method `solve` which initialize the above `backtrack` function with the appropiate initial conditions. 

In addition, we incorporate the method `print` which prints the corresponding results after solving the problem. We recommend to use this method in small instances of the problems.

#  Integer Partition Problem

For solving the *Integer Partition* problem of a number $n$ we show what we understand as a candidate `c`. We abstract the candidate as a list with three elements, for example, for number $n=5$ a possible candidate is
$$c = [[3,1],1,1],$$
where 
- The first entry $[3,1]$ represents current partition. 
- The second entry $1$ represents the difference between the target number $n=5$ and the sum of the current partition.
- The third entry represents an upper bound for the numbers that could be added to the current partition list. In this case $1$. This entry allows to keep an order of the elements added to the partition, therefore we avoid duplicates in the candidates tree exploration.

The `first` method in this case represents the following transition
$$[[3,1],1,1]\rightarrow [[3,1,1],0,1]. $$

Because $[3,1,1]$ is a valid partition we return in the function call stack saving this partition in the `output`. 

After accepting, the `next` method generate the next candidate in breadth, then we have the following transition:
$$[[3,1],1,1]\rightarrow [[3,1],1,0],$$
i.e., we quit the last added to list, update the difference with the target number $n=5$ and reduce the available upper bound for the numbers that we could add. 

As remark, in the transitions between candidates, we overwrite the information as possible in order to be efficient in memory, that is the reason we use the `backtrack` function inside the class.

The `rejection` and `accept` methods allows to backtrack to candidates where we could continue to explore, in this case, we backtrack to $[[],5,2]$. 


The name of the class implemented as above is `Partition`. In the following cell of code we solve the partitions for target number $n=5$

In [4]:
P = Partition(n = 5)
P.solve()

We print the partitions

In [5]:
P.print() 

[5]
[4, 1]
[3, 2]
[3, 1, 1]
[2, 2, 1]
[2, 1, 1, 1]
[1, 1, 1, 1, 1]


With this implementation we could calculate the partitions for $n=80$ in approximately 2 minutes.

In [6]:
n = 10 # try a big number, for n=80 the time is less than 2 minutes
P_large_instance = Partition(n)
P_large_instance.solve()

We print the number of partitions to validate

In [7]:
print(f"The number of partions of {n} is: {len(P_large_instance.partitions)}")

The number of partions of 10 is: 42


# Gray code

For solving the *Gray code* problem of $n$ bits we should show what we understand as a candidate `c`. We abstract the candidate as a list with two elements, for example, for $n=3$ bits a candidate could be 
$$c = [[0,1,3],1],$$
where 
- The first entry $[0,1,3]$ the current Gray code in decimal representation. In this case we have an incomplete code, we will add elements during the exploration. 
- The second entry $1$ represents an element in $\mathbb{Z}/n\mathbb{Z}$ we use for adding elements to the code in the methods `first` and `next`.

Let $c=[[c_1,\dots,c_{k}],r_c]$ with $r_c\in \mathbb{Z}/n\mathbb{Z}$.

The `first` method define a new candidate $s$ in depth as follows
$$\quad c_{k+1}=c_k\wedge(1<< 0),$$
where `^` is the `XOR` operator and `<<` is the left shift operator, thus $s=[[c_1,\dots,c_k,c_{k+1}],0]$

In this case, we have the following transition
$$[[0,1,3],1]\rightarrow [[0,1,3,2],0]$$
 

The `next` method generate the next candidate in breadth, then we should pop the last added element and continue with the following transition 
$$r_s\equiv r_c+1\ \text{mod }n, \quad \text{ and }\quad c_{k}=c_{k-1}\wedge(1<< r_s),$$
thus $s=[[c_1,\dots,c_k],r_s]$

In this case, a transition like `next` method is 
$$[[0,1,3],1]\rightarrow [[0,1,5],2]$$

As remark, in the transitions between candidates, we overwrite the information as possible in order to be efficient in memory, that is the reason we use the `backtrack` function inside the class.

The `rejection`,`accept` methods, and the recursion allows us to backtrack to candidates where we could continue to explore.


The name of the class implemented as above is `GrayCode`. In the following cell of code we get a Gray code for target number of bits $n=4$ 

In [8]:
G = GrayCode(4) # small example
G.solve()

We print the result for the small instance with the normal binary representation.

In [9]:
G.print()

0000
0001
0011
0010
0110
0111
0101
0100
1100
1101
1111
1110
1010
1011
1001
1000


This implementation reduce the number of recursion calls, therefore, we could do a instance bigger than $n=15$ bits. In the following code, we solve a instance of $n=17$ in 4 minutes

In [10]:
n = 15 # try a big number, for n=17 the time is approx 4 minutes, n=15 is not a problem.
G_large_instance = GrayCode(n)
G_large_instance.solve()

# Conclusions

The implementation presented here tries to reduce the number of recursion calls in order to allow big instances. The main idea was to avoid loops and try to incorporate in `first`, and `next` an efficient way to explore the candidates tree. The tradeoff was to loose intuition of how to understand the candidate in each problem. 

The general `backtrack` function is reusable and we incorporate it in the `solve` method of the classes `Partition`, and `GrayCode` in the same way, but of course we initialize in a different way.

The main reason to do that was:
- Avoid creating copies of the class, i.e., this approach is more efficient than to pass a class as an argument in a recursive function. 
- Initialize the backtracking algorithm in a correct way. 
- It is easy to use for the user.

As we could see, we reach a relative good performance in instance space for both problems using the proposed `backtrack` function. 

Points of improvements are:
- Enhance the interpretability of the exploration. 
- Explore in a more efficient way the candidates tree to reach larger instances.